# Chapter 2. Data cleansing

Now that we got the needed data, it's time to look at it to figure out whether it is ready to be used in our algorithms (spoiler: it is not). 

## Imports

At first we have some pretty standard modules here: `pandas`, `numpy` and `utils`, our file with custom functions. 

We're also going to keep our data in a .json file rather than .csv, and we'll need `json` module to perform the corresponding conversion. 

Built-in pandas functions for regular expressions will be enough for us almost everywhere, but in some cases we'll still need `re` module.

In [1]:
import pandas as pd
import numpy as np
import utils
import json
import re

## Captions transformation

The first step in our work today is to read the data from our `videos.csv` file and look at what we have there.

In [2]:
df = pd.read_csv("./data/videos.csv", index_col='Unnamed: 0')
df.drop(columns=['id'], inplace=True)

In [3]:
df.head()

,title,description,thumbnail,video_id,captions,published_at,tags,channel_id,views,likes,dislikes,comments_count
0,Your identity is your superpower | America Fer...,Watch the full talk: http://tedtalks.social/su...,https://i.ytimg.com/vi/XQJhRDbsDzI/default.jpg,XQJhRDbsDzI,"1\n00:00:00,350 --> 00:00:06,470\nWhen I was 1...",2021-07-10T14:00:12Z,"['TEDTalk', 'TEDTalks', 'TED Talk', 'TED Talks...",UCAuUUnT6oDeKwE6v1NGQxug,69496,2991.0,385.0,8.0
1,Documentary films that explore trauma -- and m...,Visit http://TED.com/shapeyourfuture to watch ...,https://i.ytimg.com/vi/xvG3ftEv5rM/default.jpg,xvG3ftEv5rM,"1\n00:00:14,331 --> 00:00:16,542\n[SHAPE YOUR ...",2021-07-09T15:30:06Z,"['TEDTalk', 'TEDTalks', 'TED Talk', 'TED Talks...",UCAuUUnT6oDeKwE6v1NGQxug,26010,1076.0,35.0,59.0
2,A cleanse won't detox your body -- but here's ...,Put down the cayenne-lemon water and step away...,https://i.ytimg.com/vi/DESCcjSQSKY/default.jpg,DESCcjSQSKY,"1\n00:00:00,000 --> 00:00:07,000\nTranscriber:...",2021-07-08T15:15:00Z,"['TEDTalk', 'TEDTalks', 'TED Talk', 'TED Talks...",UCAuUUnT6oDeKwE6v1NGQxug,457410,23220.0,452.0,828.0
3,What should humans take to space (and leave be...,Visit http://TED.com/shapeyourfuture to watch ...,https://i.ytimg.com/vi/D3LFVlI5ir8/default.jpg,D3LFVlI5ir8,"1\n00:00:14,871 --> 00:00:16,579\n[SHAPE YOUR ...",2021-07-07T15:14:57Z,"['TEDTalk', 'TEDTalks', 'TED Talk', 'TED Talks...",UCAuUUnT6oDeKwE6v1NGQxug,23259,628.0,153.0,145.0
4,How to be a professional troublemaker | Luvvie...,Visit http://TED.com to get our entire library...,https://i.ytimg.com/vi/EfeiKPSBKfg/default.jpg,EfeiKPSBKfg,"1\n00:00:00,000 --> 00:00:07,000\nTranscriber:...",2021-07-06T16:45:00Z,"['TEDTalk', 'TEDTalks', 'TED Talk', 'TED Talks...",UCAuUUnT6oDeKwE6v1NGQxug,25494,844.0,96.0,95.0


In [4]:
df.shape

(3589, 12)

The first thing I would like to cope with is subtitles, as they have a major importance for the aims of the project. Let's take a look at what we've got in the previous chapter.

In [5]:
captions = df['captions']
captions.iloc[-1]

'1\n00:00:26,000 --> 00:00:30,000\nI wrote this poem after hearing a pretty well known actress\n\n2\n00:00:30,000 --> 00:00:33,000\ntell a very well known interviewer on television,\n\n3\n00:00:33,000 --> 00:00:35,000\n"I\'m really getting into the Internet lately.\n\n4\n00:00:35,000 --> 00:00:37,000\nI just wish it were more organized."\n\n5\n00:00:37,000 --> 00:00:38,000\nSo ...\n\n6\n00:00:38,000 --> 00:00:40,000\n(Laughter)\n\n7\n00:00:42,000 --> 00:00:45,000\nIf I controlled the Internet,\n\n8\n00:00:45,000 --> 00:00:49,000\nyou could auction your broken heart on eBay.\n\n9\n00:00:49,000 --> 00:00:51,000\nTake the money; go to Amazon;\n\n10\n00:00:51,000 --> 00:00:55,000\nbuy a phonebook for a country you\'ve never been to -- call folks at random\n\n11\n00:00:55,000 --> 00:00:58,000\nuntil you find someone who flirts really well in a foreign language.\n\n12\n00:00:58,000 --> 00:00:59,000\n(Laughter)\n\n13\n00:00:59,000 --> 00:01:01,000\nIf I were in charge of the Internet,\n\n14\n

Expectedly, the captions now are a complete mess. It's not just simple pretty text data, it's made up of scary numbered lines with time codes. That's something we have to deal with, and slicing seems to be a really helpful and simple solution.

In [6]:
def transform_captions(caption):
    caption = caption.split(sep="\n")
    caption = caption[::-4]
    caption.reverse()
    caption = ' '.join(caption)
    return caption

In [7]:
captions = captions.apply(transform_captions)
print(captions.iloc[-1])

I wrote this poem after hearing a pretty well known actress tell a very well known interviewer on television, "I'm really getting into the Internet lately. I just wish it were more organized." So ... (Laughter) If I controlled the Internet, you could auction your broken heart on eBay. Take the money; go to Amazon; buy a phonebook for a country you've never been to -- call folks at random until you find someone who flirts really well in a foreign language. (Laughter) If I were in charge of the Internet, you could Mapquest your lover's mood swings. Hang left at cranky, right at preoccupied, U-turn on silent treatment, all the way back to tongue kissing and good lovin'. You could navigate and understand every emotional intersection. Some days, I'm as shallow as a baking pan, but I still stretch miles in all directions. If I owned the Internet, Napster, Monster and Friendster.com would be one big website. That way you could listen to cool music while you pretend to look for a job and you'r

Well, now it looks like a normal edible text rather than that inconvenient thing with timestamps. We can replace our `captions` column in the dataframe with what we have now.

In [8]:
df['captions'] = captions

## Video titles and speaker names
The next step that I'd take is looking at the titles. Are they in an appropriate form? Of course they are not, that would be too simple.

In [9]:
df_temp = df.copy()

In [10]:
for i in range(10):
    print(df_temp.iloc[i, 0])

Your identity is your superpower | America Ferrera
Documentary films that explore trauma -- and make space for healing | Almudena Toral
A cleanse won't detox your body -- but here's what will | Body Stuff with Dr. Jen Gunter
What should humans take to space (and leave behind)? | Jorge Mañes Rubio
How to be a professional troublemaker | Luvvie Ajayi Jones
The messy truth about grief | Nora McInerny
What's normal anxiety -- and what's an anxiety disorder? | Body Stuff with Dr. Jen Gunter
The infinite alchemy of storytelling | Zahra Al-Mahdi
The rigged test of leadership | Sophie Williams
How to heat your home without hurting the planet | Kathy Hannun


The problem with these titles is that they contain speakers' names along with the videos' "names". In order to have a clean and pretty dataset we're going to extract names into a separate column and remove them from our `title` column. That's where regular expressions come in handy.

In [11]:
vertical_bar_names = df_temp['title'].str.extract(r'\|\s*(.+)$')

Using the regular expression above we've extracted all symbols that were met after `|` symbol in our `title` column, since 10 most recent videos' titles are compiled in accordance with this scheme. Let's see if that's enough.

In [12]:
def count_extracted_names(speaker_series, df):
    print(f"Number of speaker names extracted: {speaker_series.notnull().sum().iloc[0]}/{df.shape[0]}")

In [13]:
for i in range(10):
    print(vertical_bar_names[vertical_bar_names[0].notnull()].iloc[i, 0])

America Ferrera
Almudena Toral
Body Stuff with Dr. Jen Gunter
Jorge Mañes Rubio
Luvvie Ajayi Jones
Nora McInerny
Body Stuff with Dr. Jen Gunter
Zahra Al-Mahdi
Sophie Williams
Kathy Hannun


In [14]:
count_extracted_names(vertical_bar_names, df_temp)

Number of speaker names extracted: 2126/3589


We have dealt with the most of video titles, but there are still many left. It means that not every video title corresponds to the form we have expected.  

In [15]:
df_temp['speaker_name'] = vertical_bar_names
df_temp = df_temp[df_temp['speaker_name'].isna()]

In [16]:
for i in range(10):
    print(df_temp[df_temp['speaker_name'].isna()].iloc[-i, 0])

What foods did your ancestors love? Aparna Pallavi
Dean Ornish: The world's killer diet
Richard Baraniuk on open-source learning
Wade Davis: Cultures at the far edge of the world
Bjorn Lomborg: Global priorities bigger than climate change
Phil Borges: Documenting our endangered cultures
Peter Gabriel: Fighting injustice with a videocamera
Robert Neuwirth: The "shadow cities" of the future
Kevin Kelly: How technology evolves
Peter Donnelly: How stats fool juries


The situation is getting worse: we will need a more complicated regex to cope with those names that go at the beginning of the line. But at least we see that they are usually followed by a colon.

Note the fact that we are going to apply next regex only to the rows from which we've failed to extract speaker names earlier. That is why we have created a temporary DataFrame `df_temp` and dropped the rows with non-null `speaker_name` values.

In [17]:
colon_names = df_temp['title'].str.extract(r'^((?:[A-Z]{1}[^:\(\)]+){1,}):.*$')

In [18]:
for i in range(10):
    print(colon_names[colon_names[0].notnull()].iloc[-i, 0])

Henry Lin
Dean Ornish
Wade Davis
Bjorn Lomborg
Phil Borges
Peter Gabriel
Robert Neuwirth
Kevin Kelly
Peter Donnelly
Burt Rutan


In [19]:
count_extracted_names(colon_names, df_temp)

Number of speaker names extracted: 1355/1463


In [20]:
df_temp['speaker_name'] = colon_names
df_temp = df_temp[df_temp['speaker_name'].isna()]

In [21]:
for i in range(10):
    print(df_temp.iloc[i, 0])

What foods did your ancestors love? Aparna Pallavi
The path to ending systemic racism in the US
Get TED Talks recommended just for you
Visions of Africa's future, from African filmmakers
The trauma of systematic racism is killing Black women. A first step toward change...
The top 10 TED Talks of 2016
Ideas worth dating
Chimps have feelings and thoughts. They should also have rights
TED, the Musical
The interspecies Internet? An idea in progress...


Now that we are done with our so-called colon names, we can use an apostrophe after some other names as a break point. 

In [22]:
apostrophe_names = df_temp['title'].str.extract(r'^((?:[A-Z]{1}[-\w.]+\s?)+)\'s?.*$')

In [23]:
for i in range(10):
    print(apostrophe_names[apostrophe_names[0].notnull()].iloc[-i, 0])

Shimon Schocken
Matthew Childs
Ben Katchor
Tom Shannon
Nandan Nilekani
Yves Behar
Michelle Obama
Liz Coleman
Felix Dennis
Paul Collier


In [24]:
count_extracted_names(apostrophe_names, df_temp)

Number of speaker names extracted: 21/108


In [25]:
df_temp['speaker_name'] = apostrophe_names
df_temp = df_temp[df_temp['speaker_name'].isna()]

In [26]:
for i in range(10, 20):
    print(df_temp.iloc[i, 0])

iO Tillett Wright: Fifty shades of gay
Lemon Andersen performs "Please don't take my Air Jordans"
Boaz Almog "levitates" a superconductor
Reggie Watts disorients you in the most entertaining way
Morley sings "Women of Hope"
Award-winning teen-age science in action
What we learned from 5 million books
Maya Beiser(s) and her cello(s)
Damon Horowitz calls for a "moral operating system"
Robert Gupta and Joshua Roman duet on "Passacaglia"


It's time to deal with the names that are not followed by anything at all. These may bring some mistakes, since we do not have a clear break point. The capitalization is our means to differ names from anything else. However, we will be unable to capture one-word names because literally every first word in titles is going to be capitalized.

In [27]:
no_breakpoint_names = df_temp['title'].str.extract(r'^((?:[A-Z]{1}[-a-z.]+\s?(?:and|with)?\s?){2,}).*$')

In [28]:
for i in range(10, 20):
    print(no_breakpoint_names[no_breakpoint_names[0].notnull()].iloc[-i, 0])

Nalini Nadkarni 
Jacqueline Novogratz 
Bruce Bueno 
Alex Tabarrok 
Sean Gourley 
Mae Jemison 
Louise Fresco 
Jay Walker 
Kaki King 
Yann Arthus-Bertrand 


In [29]:
count_extracted_names(no_breakpoint_names, df_temp)

Number of speaker names extracted: 68/87


In [30]:
df_temp['speaker_name'] = no_breakpoint_names
df_temp = df_temp[df_temp['speaker_name'].isna()]

Look, we have only failed to extract 19 names out of 3589!  
We can try to deal with them manually or just drop. In any case, we now need to gather up the new data in our good old `df` dataframe.

In [31]:
for i in df_temp.index:
    print(i, df_temp.loc[i, 'title'])

252 What foods did your ancestors love? Aparna Pallavi
287 The path to ending systemic racism in the US
663 Get TED Talks recommended just for you
938 Visions of Africa's future, from African filmmakers
1238 The trauma of systematic racism is killing Black women. A first step toward change...
1358 The top 10 TED Talks of 2016
1470 Ideas worth dating
1756 Chimps have feelings and thoughts. They should also have rights
2026 TED, the Musical
2191 The interspecies Internet? An idea in progress...
2306 iO Tillett Wright: Fifty shades of gay
2558 Morley sings "Women of Hope"
2561 Award-winning teen-age science in action
2629 What we learned from 5 million books
2718 9/11 healing: The mothers who found forgiveness, friendship
2957 Lies, damned lies and statistics (about TEDTalks)
3133 The design genius of Charles + Ray Eames
3164 Naturally 7 beatbox a whole band
3195 Extreme wingsuit flying (The TEDTalk)


In [32]:
names_list = [vertical_bar_names, colon_names, apostrophe_names, no_breakpoint_names]

for x in names_list:
    x.dropna(inplace=True)
    
sp = pd.concat(names_list)
df['speaker_name'] = sp

In [33]:
df.loc[252, 'speaker_name'] = 'Aparna Pallavi'
df.loc[2306, 'speaker_name'] = 'iO Tillett Wright'
df.loc[2558, 'speaker_name'] = 'Morley'
df.loc[3133, 'speaker_name'] = 'Charles + Ray Eames'

In [34]:
df = df[df['speaker_name'].notnull()]
df.reset_index(drop=True, inplace=True)
df.shape

(3574, 13)

Let's not forget that we must also remove names we've extracted from the title column along with some special characters like `|+: ` and space characters. 

In [35]:
for i in range(10):
    print(i, df.loc[i, 'title'])

0 Your identity is your superpower | America Ferrera
1 Documentary films that explore trauma -- and make space for healing | Almudena Toral
2 A cleanse won't detox your body -- but here's what will | Body Stuff with Dr. Jen Gunter
3 What should humans take to space (and leave behind)? | Jorge Mañes Rubio
4 How to be a professional troublemaker | Luvvie Ajayi Jones
5 The messy truth about grief | Nora McInerny
6 What's normal anxiety -- and what's an anxiety disorder? | Body Stuff with Dr. Jen Gunter
7 The infinite alchemy of storytelling | Zahra Al-Mahdi
8 The rigged test of leadership | Sophie Williams
9 How to heat your home without hurting the planet | Kathy Hannun


In [36]:
unique_names = list(df.speaker_name.unique())
unique_names = [x for x in unique_names if str(x) != 'nan']

for name in unique_names:
    df['title'] = df['title'].str.replace(name, '', regex=False) 

In [37]:
df['title'] = df['title'].str.replace(r'(\||\+|:)', '', regex=True)
df['title'] = df['title'].str.replace(' ?\(.*\)', '', regex=True)

df['title'] = df['title'].str.replace(r'^\'s?\s?', ' ', regex=True)

df['title'] = df['title'].str.replace(r'\s{2,}', ' ', regex=True)
df['title'] = df['title'].str.replace(r'^\s', '', regex=True)
df['title'] = df['title'].str.replace(r'\s$', '', regex=True)

In [38]:
df['speaker_name'] = df['speaker_name'].str.replace(r'\|? ted ?talks?', '', flags=re.IGNORECASE, regex=True)
df['speaker_name'] = df['speaker_name'].str.replace(r'(?: with| and | \+)', ',', regex=True)
df['speaker_name'] = df['speaker_name'].str.replace(r'(?:\(|\))', '', regex=True)

In [39]:
for i in range(3133, 3143):
    print(i, df.loc[i, 'title'])

3133 "Success is a continuous journey"
3134 Learning from the gecko's tail
3135 plugs smart power outlets
3136 invents eco-friendly drywall
3137 on the world in 2200
3138 odes to vice and consequences
3139 captures fragile Earth in wide-angle
3140 A university for the coming singularity
3141 call to reinvent liberal arts education
3142 rocks out to "Playing with Pink Noise"


Now we are officially done with speaker names and titles. And it means that it's time to move on to the next issue.

## Transcribers and reviewers

That is a small task that requires a bit more work with subtitles. There's some information about those who transcribed and rewieved the subtitles right inside the `captions` column. It is pretty improbable that we will make a good use of it, but let's extract this data just to be nice (and to have a detailed and thorough dataset).  

I think that it's easier to open a file with raw captions once again so as to have end-of-line symbols that are pretty useful for us now.

In [40]:
raw = pd.read_csv("./data/temp/raw_captions.csv", index_col='Unnamed: 0')
raw.rename(columns = {'0': 'captions'}, inplace = True)

In [41]:
transcriber = raw['captions'].str.extract(r'Transcriber: ?(.+)(?:Reviewer:.*)\n', flags=re.IGNORECASE)
reviewer = raw['captions'].str.extract(r'Transcriber:.+Reviewer: ?(.+)\n', flags=re.IGNORECASE)

In [42]:
transcriber[transcriber[0].notnull()].shape, reviewer[reviewer[0].notnull()].shape

((148, 1), (148, 1))

In [43]:
df['transcriber'] = transcriber
df['reviewer'] = reviewer

In [44]:
unique_transcribers = list(transcriber[0].unique())
unique_reviewers = list(reviewer[0].unique())
unique_transcribers.reverse(), unique_reviewers.reverse()
unique_transcribers.pop(), unique_reviewers.pop()

(nan, nan)

In [45]:
df['captions'] = df['captions'].str.replace('Transcriber:', '', flags=re.IGNORECASE)
df['captions'] = df['captions'].str.replace('Reviewer:', '', flags=re.IGNORECASE)
for name in unique_transcribers:
    df['captions'] = df['captions'].str.replace(name, '')
for name in unique_reviewers:
    df['captions'] = df['captions'].str.replace(name, '')

In [46]:
df.iloc[103:108, [0, 13, 14]]

,title,transcriber,reviewer
103,The joy of shopping -- and how to recapture it...,Leslie Gauthier,Joanna Pietrulewicz
104,Community-powered solutions to the climate crisis,TED Translators Admin,Mirjana Čutura
105,How your brain responds to stories -- and why ...,Ivana Korom,Joanna Pietrulewicz
106,How a strong creative industry helps economies...,Joseph Geni,Camille Martínez
107,4 tips to kickstart honest conversations at work,Leslie Gauthier,Joanna Pietrulewicz


## Working with descriptions

Now it's time for some tedious regular expression work with video descriptions. As you can see below, the descriptions contain many chunks of information that are either non-unique or just are not related to the topic of the video directly. 

In [47]:
for i in range(3):
    print(df.iloc[i, 1])
    print("\n--------------------------\n")

Watch the full talk: http://tedtalks.social/superpower

A clip from America Ferrera's TED Talk "My identity is a superpower -- not an obstacle" from TED2019

Hollywood needs to stop resisting what the world actually looks like, says actor, director and activist America Ferrera. Tracing the contours of her career, she calls for more authentic representation of different cultures in media -- and a shift in how we tell our stories. "Presence creates possibility," she says. "Who we see thriving in the world teaches us how to see ourselves, how to think about our own value, how to dream about our futures."

The TED Talks channel features the best talks and performances from the TED Conference, where the world's leading thinkers and doers give the talk of their lives in 18 minutes (or less). Look for talks on Technology, Entertainment and Design -- plus science, business, global issues, the arts and more. You're welcome to link to or embed these videos, forward them to others and share these

In [48]:
df_checkpoint = df.copy()

Here we should take a look at different variations of paragraphs that are useless for us and create some regexes in order to delete them from our dataset. 

In [49]:
df['description'] = df['description'].str.replace(r'Visit http://TED\.com[^\n\n]*', '', \
                                                  flags=re.IGNORECASE, regex=True)
df['description'] = df['description'].str.replace(r'TEDTalks is a daily video podcast [^\n\n]*', '', \
                                                  flags=re.IGNORECASE, regex=True)
df['description'] = df['description'].str.replace(r'[^\n\n]*TED Talks channel features [^\n\n]*', '', \
                                                  flags=re.IGNORECASE, regex=True)
df['description'] = df['description'].str.replace(r'note: comments [^\n\n]*', '', \
                                                  flags=re.IGNORECASE, regex=True)
df['description'] = df['description'].str.replace(r'[^\n\n]* CC button [^\n\n]*', '', regex=True)
df['description'] = df['description'].str.replace(r'[^\n\n]*Watch the full talk[^\n\n]*', '', regex=True)
df['description'] = df['description'].str.replace('http://www.ted.com', '', regex=False)
df['description'] = df['description'].str.replace(r'[^\n]*https?://[^\n]*', '', regex=True)
df['description'] = df['description'].str.replace(r'[^\n]*\d{1,2}:\d{2}[^\n]*', '', regex=True)
df['description'] = df['description'].str.replace(r'[^\n]*A clip from[^\n]*', '', regex=True)
df['description'] = df['description'].str.replace(r'[^\n]*Think you know how your body works?[^\n]*', '', regex=True)
df['description'] = df['description'].str.replace(r'Find closed captions [^\n\n]*', '', regex=True)
df['description'] = df['description'].str.replace(r'\(?NOTE [^\n\n]*', '', \
                                                  flags=re.IGNORECASE, regex=True)
df['description'] = df['description'].str.replace('Take action on climate change at http://countdown.ted.com.', '', \
                                                  flags=re.IGNORECASE, regex=False)
df['description'] = df['description'].str.replace('Check out more TED Talks:', '', \
                                                  regex=False, flags=re.IGNORECASE)
df['description'] = df['description'].str.replace('/index.php/talks/top10', '', \
                                                  regex=False, flags=re.IGNORECASE)
df['description'] = df['description'].str.replace('www.ted.com', '', \
                                                  regex=False, flags=re.IGNORECASE)

We would also like to perform some operations with repetitive spaces and new-line symbols, or those that go in the end or beginning.  
It will make our data clearer.

In [50]:
columns = ['speaker_name', 'transcriber', 'reviewer', 'captions', 'description']

for col in columns:
    df[col] = df[col].str.replace('\n{2,}', '\n', regex=True)
    df[col] = df[col].str.replace('\n+$', '', regex=True)
    df[col] = df[col].str.replace('^\n+', '', regex=True)
    df[col] = df[col].str.replace('\s{2,}', ' ', regex=True)
    df[col] = df[col].str.replace('^\s', '', regex=True)
    df[col] = df[col].str.replace('\s$', '', regex=True)

In [51]:
for i in range(3):
    print(df.iloc[i, 1])
    print("\n--------------------------\n")

Hollywood needs to stop resisting what the world actually looks like, says actor, director and activist America Ferrera. Tracing the contours of her career, she calls for more authentic representation of different cultures in media -- and a shift in how we tell our stories. "Presence creates possibility," she says. "Who we see thriving in the world teaches us how to see ourselves, how to think about our own value, how to dream about our futures."

--------------------------

Through documentary films following survivors of trauma, TED Fellow Almudena Toral makes invisible psychological scars seen. She shares the heartbreaking story of Adayanci Pérez, a six-year-old girl from Guatemala suffering from post-traumatic stress disorder (PTSD) due to her encounter with US immigration enforcement. A powerful call to give voice to those who are silenced -- and pressure governments to change their course of action.

--------------------------

Put down the cayenne-lemon water and step away from 

Here are the results of our work! Now the descriptions look informative and tidy.

## Extracting date and time

We are also going to do something with our `published_at` column to make it easier to work with.

In [52]:
df.published_at[0], df.published_at[2442]

('2021-07-10T14:00:12Z', '2012-06-19T18:00:04Z')

Fortunately, date and time of video publication are presented in a constant form, which means that regexes will save our time and sanity once again.

In [53]:
datetime_r = r"^(\d{4})-(\d{2})-(\d{2})T(\d{2}):(\d{2}):\d{2}Z$"
date_time = df['published_at'].str.extract(datetime_r)

In [54]:
date_time.rename(columns={0:'year', 1:'month', 2:'day', 3:'hour', 4:'minute'}, inplace=True)
df = pd.concat([df, date_time], axis=1)
df.drop(columns='published_at', inplace=True)

In [55]:
df.iloc[:3, [0, 14, 15, 16, 17, 18]]

,title,year,month,day,hour,minute
0,Your identity is your superpower,2021,07,10,14,00
1,Documentary films that explore trauma -- and m...,2021,07,09,15,30
2,A cleanse won't detox your body -- but here's ...,2021,07,08,15,15


This is it, really simple after everything we have done previously.

## Json conversion

The last thing to do today is going to be some tags work. But right before that we need to transform our Pandas DataFrame into a .json object so as to preserve the dictionary structure of the data.

In [56]:
df = df[['video_id', 'channel_id', 'title', 'speaker_name', 'year', 'month', 'day', 'hour', 'minute', 'description', \
         'thumbnail', 'captions', 'transcriber', 'reviewer', 'views', 'likes', 'dislikes', 'comments_count', 'tags']]

In [57]:
df.to_csv('./data/videos_cleansed.csv')
utils.upload_to_googledrive('videos_cleansed.csv')

In [58]:
data_json = df.to_json(orient="records")
data_json = json.loads(data_json)

In [59]:
pd.json_normalize(data_json, max_level=0).tail(2)

,video_id,channel_id,title,speaker_name,year,month,day,hour,minute,description,thumbnail,captions,transcriber,reviewer,views,likes,dislikes,comments_count,tags
3572,Y6bbMQXQ180,UCAuUUnT6oDeKwE6v1NGQxug,"Secrets of success in 8 words, 3 minutes",Richard St. John,2007,01,06,19,53,Why do people succeed? Is it because they're s...,https://i.ytimg.com/vi/Y6bbMQXQ180/default.jpg,This is really a two-hour presentation I give ...,None,None,2767300,39311.0,635.0,645.0,"['Richard St. John', 'Richard John', 'TED', 'T..."
3573,gu_PQBmk-6c,UCAuUUnT6oDeKwE6v1NGQxug,If I controlled the Internet,Rives,2006,12,25,17,58,"How many poets could cram eBay, Friendster and...",https://i.ytimg.com/vi/gu_PQBmk-6c/default.jpg,I wrote this poem after hearing a pretty well ...,None,None,351573,3362.0,174.0,362.0,"['Rives', 'TED', 'TEDTalks', 'talks', 'spoken ..."


By the way, we are still able to normalize our json object back to a dataframe for better visibility (if we need).

## Tag labelling

Finally, it's time to make dictionaries out of our tags. These dictionaries will contain not only tags themselves, but their types as well. Tags that contain something about "ted talks" will be labeled as garbage as they don't actually tell us anything important about the video, and tags that contain names of the speakers will be labeled as "name" tags.

In [60]:
all_tags = []

for i in range(df.shape[0]):
    tags = data_json[i]['tags'].strip('\'][').split('\', \'')
    tags_list = []
    full_name = (data_json[i]['speaker_name']).split()
    
    for tag in tags:
        
        any_ted = re.findall(r'.*ted.*', tag, re.IGNORECASE)
        any_talk = re.findall(r'.*talks?.*', tag, re.IGNORECASE)
        
        tag_type = 'normal'

        if (len(any_ted) != 0) | (len(any_talk) != 0):
            tag_type = 'garbage'    
        else: 
            for name in full_name:
                any_name = re.findall(name, tag)
                if (len(any_name) != 0):
                    tag_type = 'name'                
        
        tag_dict = {'tag': tag, 'type': tag_type}
        tags_list.append(tag_dict)
        
    all_tags.append(tags_list)

In [61]:
for i in range(len(all_tags)):
    data_json[i]['tags'] = all_tags[i]

In [62]:
print(f"{data_json[3441]['speaker_name']}, '{data_json[3441]['title']}'")
data_json[3441]['tags']

Maira Kalman, 'The illustrated woman'


[{'tag': 'Maira', 'type': 'name'},
 {'tag': 'Kalman', 'type': 'name'},
 {'tag': 'TED', 'type': 'garbage'},
 {'tag': 'TEDTalks', 'type': 'garbage'},
 {'tag': 'Talks', 'type': 'garbage'},
 {'tag': 'The New Yorker', 'type': 'normal'},
 {'tag': 'illustration', 'type': 'normal'},
 {'tag': 'imagination', 'type': 'normal'}]

As you can observe, the work is done perfectly: tags connected with TED Talks were labeled as "garbage", speakers were labeled as "name", and everything else was claimed to be "normal".

## Making a dictionary out of date and time

Since we now keep our data in a json object, that would be logical to transform anything we can into dictionaries. It seems pretty natural to keep everything we know about date and time in the same place.

In [63]:
df = pd.json_normalize(data_json, max_level=0)
date_list = []

for i in range(df.shape[0]):
    year = df.iloc[i, 4]
    month = df.iloc[i, 5]
    day = df.iloc[i, 6]
    hour = df.iloc[i, 7]
    minute = df.iloc[i, 8]
    
    one_date = {'year': year, 'month': month, 'day': day,
               'hour': hour, 'minute': minute}
    date_list.append(one_date)
    
df['published_at'] = date_list
df.drop(columns=['year', 'month', 'day', 'hour', 'minute'], inplace=True)
df = df[['video_id', 'channel_id', 'title', 'speaker_name', 'published_at', 'description', \
         'thumbnail', 'captions', 'transcriber', 'reviewer', 'views', 'likes', 'dislikes', 'comments_count', 'tags']]

data_json = df.to_json(orient="records")
data_json = json.loads(data_json)

In [64]:
data_json[3573]

{'video_id': 'gu_PQBmk-6c',
 'channel_id': 'UCAuUUnT6oDeKwE6v1NGQxug',
 'title': 'If I controlled the Internet',
 'speaker_name': 'Rives',
 'published_at': {'year': '2006',
  'month': '12',
  'day': '25',
  'hour': '17',
  'minute': '58'},
 'description': "How many poets could cram eBay, Friendster and Monster.com into 3-minute poem worthy of a standing ovation? Enjoy Rives' unique talent.",
 'thumbnail': 'https://i.ytimg.com/vi/gu_PQBmk-6c/default.jpg',
 'captions': 'I wrote this poem after hearing a pretty well known actress tell a very well known interviewer on television, "I\'m really getting into the Internet lately. I just wish it were more organized." So ... (Laughter) If I controlled the Internet, you could auction your broken heart on eBay. Take the money; go to Amazon; buy a phonebook for a country you\'ve never been to -- call folks at random until you find someone who flirts really well in a foreign language. (Laughter) If I were in charge of the Internet, you could Mapques

In [65]:
with open('./data/videos_clean_tags.json', 'w') as file:
    json.dump(data_json, file)

utils.upload_to_googledrive('videos_clean_tags.json')

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3574 entries, 0 to 3573
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   video_id        3574 non-null   object 
 1   channel_id      3574 non-null   object 
 2   title           3574 non-null   object 
 3   speaker_name    3574 non-null   object 
 4   published_at    3574 non-null   object 
 5   description     3574 non-null   object 
 6   thumbnail       3574 non-null   object 
 7   captions        3574 non-null   object 
 8   transcriber     148 non-null    object 
 9   reviewer        148 non-null    object 
 10  views           3574 non-null   int64  
 11  likes           3564 non-null   float64
 12  dislikes        3564 non-null   float64
 13  comments_count  3538 non-null   float64
 14  tags            3574 non-null   object 
dtypes: float64(3), int64(1), object(11)
memory usage: 419.0+ KB


Now that we are done with data cleansing, it is time to move on to other kinds of tasks.